# ReAct Pattern with OpenAI

This notebook demonstrates a **ReAct (Reason + Act) pattern** for an **airline refund** scenario, **using OpenAI** for the LLM calls:

1. **Policy Logic** – Contains the airline’s refund rules.
2. **OpenAI Adapter** – Our function to call OpenAI’s API (instead of a mock).
3. **ReAct Agent** – Implements the Reason → Act → Observe → Conclude loop.
4. **Main Orchestration** – Ties it all together to decide whether the passenger is eligible for a refund.

We'll pass short prompts to OpenAI that mimic "thinking" (Reason) or "tool usage" (Act), and parse the results.

## Requirements
- `openai` Python package (`pip install openai`).
- An **OpenAI API key** set in your environment (e.g., `OPENAI_API_KEY`).

## Outline
1. Policy logic for refunds.
2. OpenAI LLM adapter.
3. ReAct-based agent.
4. A main cell to run the scenario.


## 1. Policy Logic

In [ ]:
from openai import OpenAI
import os
import re
from datetime import datetime, timedelta

os.environ['OPENAI_API_KEY']=''



AIRLINE_POLICIES = {
    "REFUND_WINDOW_HOURS": 24,
    "NON_REFUNDABLE_CLASSES": ["BasicEconomy"],
    "PARTIAL_REFUND_CLASSES": ["Economy", "PremiumEconomy"],
    "FULL_REFUND_CLASSES": ["Business", "First"]
}

def check_refund_eligibility(booking_time, cabin_class):
    """
    Check if the flight is eligible for a refund based on booking time and cabin class.
    Returns (eligible: bool, refund_amount: float)
    """
    now = datetime.now()
    time_since_booking = (now - booking_time).total_seconds() / 3600.0

    # 1) Full refund if within 24 hours
    if time_since_booking <= AIRLINE_POLICIES["REFUND_WINDOW_HOURS"]:
        return True, 1.0

    # 2) Check class rules
    if cabin_class in AIRLINE_POLICIES["NON_REFUNDABLE_CLASSES"]:
        return False, 0.0
    elif cabin_class in AIRLINE_POLICIES["FULL_REFUND_CLASSES"]:
        return True, 1.0
    elif cabin_class in AIRLINE_POLICIES["PARTIAL_REFUND_CLASSES"]:
        return True, 0.5

    return False, 0.0

## 2. OpenAI LLM Adapter
We define a **`call_openai`** function that:
- Sends a **prompt** to OpenAI (using the `gpt-3.5-turbo` model).
- Returns the **text** from the model's response.

We keep it simple. In a real application, you'd incorporate more robust logic (e.g., using function calling, better prompt engineering, etc.).

In [ ]:
def call_openai(prompt, temperature=0.2):
    """
    Calls the OpenAI ChatCompletion API with the given prompt.
    Returns the response text.
    """
    system_message = "You are a helpful AI for airline refunds."

    client = OpenAI()

    response =client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        max_tokens=150,
    )

    content = response.choices[0].message.content
    return content.strip()

## 3. ReAct Agent
We implement the same **ReAct** flow:
1. **Reason** – e.g., “Step 1: we need to check airline policy.”
2. **Act** – call the `check_refund_eligibility` tool.
3. **Observe** – see the result of that tool.
4. **Reason again** – possibly finalize.
5. **Conclude** – produce the final user-facing result.

Here, we show short prompts to OpenAI that represent these steps. In a real system, you might structure these carefully or parse partial chain-of-thought. For demonstration, we keep it simple.

In [ ]:
class ReActAirlineAgent:
    def __init__(self):
        self.conversation_log = []

    def handle_refund_request(self, booking_time, cabin_class):
        """
        Demonstrates ReAct with OpenAI calls.
        1) reason (short prompt)
        2) call tool
        3) feed observation, reason again
        4) produce final answer
        """
        # 1) Reason step with the LLM
        reason_prompt = (
            "The user wants a refund.\n"
            "Step 1: We need to check airline policy.\n"
            "Action: CallPolicyTool\n"
        )
        thought_response = call_openai(reason_prompt)
        self.conversation_log.append(f"Thought: {thought_response}")

        # 2) Actually call the policy function
        eligible, ratio = check_refund_eligibility(booking_time, cabin_class)
        observation_text = f"Observation: policy says eligible={eligible}, ratio={ratio}"
        self.conversation_log.append(observation_text)

        # 3) Provide observation to LLM and ask for conclusion
        second_prompt = (
            "We have the following observation from the policy tool:\n"
            f"{observation_text}\n"
            "Now decide the final outcome.\n"
            "Action: SummarizeRefund\n"
        )
        second_response = call_openai(second_prompt)
        self.conversation_log.append(f"Thought: {second_response}")

        # 4) Formulate final user-facing answer
        if eligible:
            refund_amount = f"{int(ratio * 100)}%"
            final_decision = f"Approved for a refund of {refund_amount} of the ticket price."
        else:
            final_decision = "Not eligible for a refund."

        # We might do one more LLM call to get a stylized message, but let's keep it simple.

        user_answer = (
            "Thank you for contacting Airline Support.\n"
            f"After reviewing your booking, you are: {final_decision}\n"
        )

        return user_answer

## 4. Main Orchestration
We simulate a user booking 30 hours ago in **Economy** class. We'll see how the ReAct agent interacts with OpenAI for reasoning steps, calls the policy function, and returns a final outcome.

Change the booking time or cabin class to see different results.

In [ ]:
if __name__ == "__main__":
    # E.g., booked 30 hours ago
    booking_time = datetime.now() - timedelta(hours=30)
    cabin_class = "Economy"

    agent = ReActAirlineAgent()
    final_msg = agent.handle_refund_request(booking_time, cabin_class)
    print("=== FINAL USER-FACING MESSAGE ===")
    print(final_msg)

    # If you'd like to see the agent's internal conversation:
    # for log_entry in agent.conversation_log:
    #     print(log_entry)

## Running the Notebook
1. **Set your OpenAI key** (e.g., `export OPENAI_API_KEY=sk-...`) or add `openai.api_key = "YOUR_KEY"` in code.
2. **Run all cells**.
3. Observe the final user-facing message.

You can also inspect the `agent.conversation_log` to see the partial LLM responses that represent "thinking."